In [28]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score
import api_call as api

In [29]:
loaded_model = joblib.load('../dump/random_forest_model.pkl')

In [30]:

# Load the new data
new_data = pd.read_csv('../data/price_2.csv') 

# Grab the new features
new_features = new_data[['RSI', 'k_percent', 'r_percent', 'Price_Rate_Of_Change', 'MACD', 'On Balance Volume']] 

# Make a prediction
new_data['Model_Prediction'] = loaded_model.predict(new_features)

# Print the first 50 rows
print(new_data[['datetime', 'symbol', 'Close', 'Prediction', 'Model_Prediction']].head(50))


      datetime symbol       Close  Prediction  Model_Prediction
0   2023-01-23    AMD   76.529999         1.0               1.0
1   2023-01-24    AMD   74.699997        -1.0              -1.0
2   2023-01-25    AMD   74.910004         1.0              -1.0
3   2023-01-26    AMD   75.160004         1.0              -1.0
4   2023-01-27    AMD   75.400002         1.0              -1.0
5   2023-01-30    AMD   72.449997        -1.0              -1.0
6   2023-01-31    AMD   75.150002         1.0              -1.0
7   2023-02-01    AMD   84.639999         1.0               1.0
8   2023-02-02    AMD   88.309998         1.0               1.0
9   2023-02-03    AMD   86.089996        -1.0              -1.0
10  2023-02-06    AMD   83.680000        -1.0              -1.0
11  2023-02-07    AMD   85.910004         1.0              -1.0
12  2023-02-08    AMD   84.690002        -1.0              -1.0
13  2023-02-09    AMD   83.209999        -1.0              -1.0
14  2023-02-10    AMD   81.480003       

In [31]:
# Calculate the accuracy
accuracy = accuracy_score(new_data['Prediction'], new_data['Model_Prediction'])
print('Accuracy: {} %'.format(accuracy*100))

Accuracy: 65.79804560260585 %


In [32]:
def execute_trade():
    # Grab the last row of the data frame
    # Filter the DataFrame to include rows with the latest date and ticker symbol
    last_row = new_data[(new_data['datetime'] == new_data['datetime'].max())]

    # Print the filtered row
    # return last_row.head()
    
    
    print("Signals for today are: ")
    for index, row in last_row.iterrows():
        print(row['symbol']," : ", row['Model_Prediction'])
    
    print("\nExecuting the following trades: ")
    #If the prediction is 1, then buy
    for index, row in last_row.iterrows():
        if row['Model_Prediction'] == 1:
            print('Buying', row['symbol'])
            api.order(row['symbol'])
        else:
            print('Selling',row['symbol'])
            api.sell(row['symbol'])
    
    print("\n")
    print("The detailed information for the trades as follows :")
    return last_row.head()
execute_trade()

Signals for today are: 
AMD  :  1.0
GOOG  :  1.0
META  :  -1.0

Executing the following trades: 
Buying AMD


Buying GOOG
Selling META


The detailed information for the trades as follows :


,Unnamed: 0,datetime,symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits,...,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Prediction,Model_Prediction
306,639,2024-04-11,AMD,167.550003,170.949997,166.550003,170.500000,48932900.0,0.0,0.0,...,164.000000,187.240005,27.969012,-72.030988,-4.155339,-2.624385,-0.055349,2.177355e+09,1.0,1.0
613,319,2024-04-11,GOOG,158.339996,161.119995,157.929993,160.789993,17832800.0,0.0,0.0,...,148.800003,161.119995,97.321412,-2.678588,3.966196,3.374363,0.056023,6.931489e+08,1.0,1.0
920,959,2024-04-11,META,521.109985,523.859985,517.289978,523.159973,10325700.0,0.0,0.0,...,481.779999,531.489990,83.242771,-16.757229,10.568882,9.841031,0.077392,8.539015e+08,1.0,-1.0
